In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,49322
2,Huelva,Confirmados PDIA 14 días,1812
3,Huelva,Tasa PDIA 14 días,"353,09936278426255"
4,Huelva,Confirmados PDIA 7 días,1170
5,Huelva,Tasa PDIA 7 dias,"227,99462166533507"
6,Huelva,Total Confirmados,49530
7,Huelva,Curados,46351
8,Huelva,Fallecidos,424


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  49322.0


/tmp/ipykernel_27254/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  13772.0


/tmp/ipykernel_27254/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_27254/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_27254/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_27254/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 373 personas en los últimos 7 días 

Un positivo PCR cada 237 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,49322.0,1170.0,1812.0,513170.0,227.994622,353.099363,341.0
Huelva-Costa,29097.0,664.0,1009.0,289548.0,229.322945,348.474174,183.0
Huelva (capital),13772.0,386.0,606.0,143837.0,268.359323,421.310233,106.0
Condado-Campiña,15339.0,297.0,525.0,156231.0,190.103117,336.040863,95.0
Sierra de Huelva-Andévalo Central,4455.0,207.0,275.0,67391.0,307.162678,408.066359,63.0
Aracena,783.0,98.0,106.0,8255.0,1187.159297,1284.070260,34.0
Almonte,2463.0,60.0,90.0,24507.0,244.828008,367.242012,14.0
Ayamonte,2162.0,30.0,36.0,21104.0,142.153146,170.583776,14.0
Cartaya,2329.0,42.0,76.0,20083.0,209.132102,378.429518,14.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puerto Moral,14.0,5.0,9.0,278.0,1798.561151,3237.410072,0.0
Cumbres Mayores,169.0,27.0,31.0,1749.0,1543.739280,1772.441395,5.0
Aracena,783.0,98.0,106.0,8255.0,1187.159297,1284.070260,34.0
Paterna del Campo,284.0,9.0,36.0,3457.0,260.341336,1041.365346,0.0
Villanueva de las Cruces,25.0,4.0,4.0,387.0,1033.591731,1033.591731,1.0
Corteconcepción,51.0,2.0,4.0,536.0,373.134328,746.268657,0.0
Fuenteheridos,19.0,4.0,5.0,709.0,564.174894,705.218618,2.0
Bollullos Par del Condado,1447.0,43.0,90.0,14387.0,298.880934,625.564746,13.0
Punta Umbría,1382.0,50.0,85.0,15355.0,325.626832,553.565614,12.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Paterna del Campo,284.0,9.0,36.0,3457.0,260.341336,1041.365346,0.0,0.250000
Jabugo,89.0,1.0,3.0,2260.0,44.247788,132.743363,0.0,0.333333
Higuera de la Sierra,75.0,1.0,3.0,1291.0,77.459334,232.378002,0.0,0.333333
Cerro de Andévalo (El),202.0,1.0,3.0,2327.0,42.973786,128.921358,0.0,0.333333
Manzanilla,155.0,3.0,8.0,2118.0,141.643059,377.714825,2.0,0.375000
Lucena del Puerto,375.0,5.0,11.0,3261.0,153.327200,337.319841,1.0,0.454545
Bollullos Par del Condado,1447.0,43.0,90.0,14387.0,298.880934,625.564746,13.0,0.477778
Palos de la Frontera,1231.0,26.0,54.0,11742.0,221.427355,459.887583,11.0,0.481481
Valverde del Camino,937.0,27.0,56.0,12750.0,211.764706,439.215686,8.0,0.482143
